In [1]:
# Load libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pySuStaIn
import statsmodels.formula.api as smf
from scipy import stats
import sklearn.model_selection

/Users/darenma/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Get neg-Z-scores

In [18]:
path = "/Users/darenma/Downloads/ADNI_Regional/"
file_z = "ADNI_neg_zscores_orig.csv"
file_d = "Regional_Tau_w_Demographics.csv"
data = pd.read_csv(path + file_z).rename(columns={"merge_RID":"RID"}).copy()
demog = pd.read_csv(path + file_d)[['RID', 'AGE']].copy()

In [31]:
df = pd.concat([data, demog], axis=1).loc[~df.AGE.isna()].copy()

In [24]:
df.head().to_csv(path+"smp.csv")

In [32]:
df.head()

,RID,merge_EXAMDATE,merge_DX,ctx-rh-bankssts,ctx-rh-caudalanteriorcingulate,ctx-rh-caudalmiddlefrontal,ctx-rh-cuneus,ctx-rh-entorhinal,ctx-rh-fusiform,ctx-rh-inferiorparietal,...,Right-Thalamus-Proper,Right-Caudate,Right-Putamen,Right-Pallidum,Right-Hippocampus,Right-Amygdala,Right-Accumbens-area,Right-VentralDC,RID,AGE
0,21,2017-11-27,CN,0.469624,0.159436,1.004761,1.185022,1.823036,0.566745,-0.086053,...,0.488576,0.789907,0.543783,2.856113,0.051000,1.224290,-0.162489,1.252460,21,72.6
1,31,2018-04-17,CN,0.579202,0.329224,1.138360,-0.354695,2.505705,1.056912,0.024999,...,1.096379,-2.137943,-1.716294,-0.521151,2.465140,2.135021,1.186946,1.164347,31,77.7
2,31,2019-04-23,CN,0.579202,0.329224,1.138360,-0.354695,2.505705,1.056912,0.024999,...,1.096379,-2.137943,-1.716294,-0.521151,2.465140,2.135021,1.186946,1.164347,31,77.7
3,56,2017-11-28,MCI,0.461606,1.035544,1.424500,1.047686,0.667291,-0.090758,0.922529,...,0.286360,0.856330,0.489305,0.962873,0.549217,0.639905,1.115385,0.793344,56,69.6
4,56,2019-01-10,MCI,0.461606,1.035544,1.424500,1.047686,0.667291,-0.090758,0.922529,...,0.286360,0.856330,0.489305,0.962873,0.549217,0.639905,1.115385,0.793344,56,69.6


In [34]:
df

,RID,merge_EXAMDATE,merge_DX,ctx-rh-bankssts,ctx-rh-caudalanteriorcingulate,ctx-rh-caudalmiddlefrontal,ctx-rh-cuneus,ctx-rh-entorhinal,ctx-rh-fusiform,ctx-rh-inferiorparietal,...,Right-Thalamus-Proper,Right-Caudate,Right-Putamen,Right-Pallidum,Right-Hippocampus,Right-Amygdala,Right-Accumbens-area,Right-VentralDC,RID,AGE
0,21,2017-11-27,CN,0.469624,0.159436,1.004761,1.185022,1.823036,0.566745,-0.086053,...,0.488576,0.789907,0.543783,2.856113,0.051000,1.224290,-0.162489,1.252460,21,72.6
1,31,2018-04-17,CN,0.579202,0.329224,1.138360,-0.354695,2.505705,1.056912,0.024999,...,1.096379,-2.137943,-1.716294,-0.521151,2.465140,2.135021,1.186946,1.164347,31,77.7
2,31,2019-04-23,CN,0.579202,0.329224,1.138360,-0.354695,2.505705,1.056912,0.024999,...,1.096379,-2.137943,-1.716294,-0.521151,2.465140,2.135021,1.186946,1.164347,31,77.7
3,56,2017-11-28,MCI,0.461606,1.035544,1.424500,1.047686,0.667291,-0.090758,0.922529,...,0.286360,0.856330,0.489305,0.962873,0.549217,0.639905,1.115385,0.793344,56,69.6
4,56,2019-01-10,MCI,0.461606,1.035544,1.424500,1.047686,0.667291,-0.090758,0.922529,...,0.286360,0.856330,0.489305,0.962873,0.549217,0.639905,1.115385,0.793344,56,69.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,6704,2019-04-16,CN,0.883881,-0.920418,0.881133,-0.037466,0.667291,1.499720,1.208417,...,1.155310,1.055598,0.999845,1.219249,0.345314,-0.365694,0.297545,0.760882,6704,65.3
815,6705,2019-04-23,Dementia,-0.014122,0.689176,0.974851,0.436441,2.442827,1.588124,-0.284808,...,0.838698,0.192101,0.859758,0.080347,2.095805,1.193932,1.442520,1.196810,6705,74.9
816,6713,2019-04-29,Dementia,-0.393635,0.845381,-0.588450,0.217863,0.616390,0.741184,-0.867974,...,0.131520,0.566167,0.081495,-0.422545,0.129868,0.268023,-0.039813,-0.372995,6713,71.1
817,6721,2019-05-07,Dementia,-0.439070,0.098312,-0.884560,0.306842,0.520577,-0.188633,-0.362260,...,1.103312,-1.194039,-0.495977,0.169092,3.292296,-0.016580,-0.990551,1.498249,6721,77.9


In [35]:
from sklearn.linear_model import LinearRegression
# df should have neg_zscore_orig.csv data plus age.

# Extracting the columns for regression analysis
X = df[['AGE']]  # Independent variable
y = df['Right-Hippocampus']  # Dependent variable

# Fitting the simple linear regression model
model = LinearRegression()
model.fit(X, y)

# Predicting the 'Right-Hippocampus' values using the fitted model
y_pred = model.predict(X)

# Calculating residuals
residuals = y - y_pred

# Adding the residuals as a new column named 'W-RHV' to the dataframe
df['W-RHV'] = residuals




,AGE,Right-Hippocampus,W-RHV
0,72.6,0.051000,-0.446825
1,77.7,2.465140,1.628257
2,77.7,2.465140,1.628257
3,69.6,0.549217,0.250839
4,69.6,0.549217,0.250839


In [36]:
# Extracting the columns for regression analysis
X = df[['AGE']]  # Independent variable
y = df['Left-Hippocampus']  # Dependent variable

# Fitting the simple linear regression model
model = LinearRegression()
model.fit(X, y)

# Predicting the 'Right-Hippocampus' values using the fitted model
y_pred = model.predict(X)

# Calculating residuals
residuals = y - y_pred

# Adding the residuals as a new column named 'W-RHV' to the dataframe
df['W-LHV'] = residuals

# Display the first few rows of the updated dataframe to verify the new column
df[['AGE', 'Right-Hippocampus', 'W-LHV', 'W-RHV']].head()


,AGE,Right-Hippocampus,W-LHV,W-RHV
0,72.6,0.051000,-0.438525,-0.446825
1,77.7,2.465140,0.788559,1.628257
2,77.7,2.465140,0.788559,1.628257
3,69.6,0.549217,0.552543,0.250839
4,69.6,0.549217,0.552543,0.250839


In [38]:
df.to_csv(path+"residual_neg_z_scores.csv")